In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import plotly.figure_factory as ff

gas_analysis_data = pd.read_csv("gas_analysis.csv")
contract_chunks_base_data = pd.read_csv("contracts_chunks_stats.csv")

In [ ]:
# Check that all tx aggregation of gas per chunk matches gas_analysis_data (it must)
tx_sum_gas = contract_chunks_base_data.groupby("tx").aggregate({"gas_used": "sum"}).reset_index()
diff = tx_sum_gas.merge(gas_analysis_data[["tx", "31bytechunker_gas"]], on="tx")
if ((diff['gas_used'] - diff['31bytechunker_gas']) != 0).any():
    raise Exception("chunks total gas doesn't match 31bytechunker gas")

In [ ]:
WITNESS_BRANCH_COST = 1900

# Create dataframe with: tx, num_contracts, avg_bytes_used_per_chunk, code_gas_used, charged_branch_cost_count, receipt_gas
df = contract_chunks_base_data.groupby("tx")
df = df.aggregate({"contract_addr": 'nunique', "bytes_used": 'mean', "gas_used": ['sum', lambda x: (x > WITNESS_BRANCH_COST).sum()]}).reset_index()

df2 = pd.DataFrame()
df2['tx'] = df['tx']
df2['num_called_contracts'] = df[('contract_addr', 'nunique')]
df2['avg_bytes_used_per_chunk'] = df[('bytes_used', 'mean')]
df2['code_gas_used'] = df[('gas_used', 'sum')]
df2['num_charged_branch_cost'] = df[('gas_used', '<lambda_0>')]

fig = px.density_heatmap(
    df2, 
    title="Called contracts vs charged WITNESS_BRANCH_COST",
    x="num_called_contracts", 
    y="num_charged_branch_cost", 
    marginal_x="histogram", 
    marginal_y="histogram", 
    nbinsx=20, 
    nbinsy=20,
    text_auto=True,
    width=1500,
    height=1000)
fig.show()

print("Uniswap tx ordered desc on number of branch cost charged:")
jj = df2.sort_values(by=["num_charged_branch_cost"], ascending=False)
print(jj[['tx', 'num_charged_branch_cost']].head().to_markdown())

q = gas_analysis_data["num_exec_contracts"].quantile(0.99)
df = gas_analysis_data[gas_analysis_data["num_exec_contracts"] <= q]

fig = go.Figure()
fig.add_trace(go.Histogram(x=df[df['to'] != contract_chunks_base_data.loc[0]['to']]['num_exec_contracts'], histnorm="percent", name="mainnet (without uniswap)"))
fig.add_trace(go.Histogram(x=df[df['to'] == contract_chunks_base_data.loc[0]['to']]['num_exec_contracts'], histnorm="percent", name="uniswap"))
fig.update_layout(barmode='overlay', width=800, height=450)
fig.update_traces(opacity=0.50)
fig.show()